In [ ]:
SOURCE_VIDEO = "/Users/farhanfadillahr/Documents/data/Kerja/yolov8/assets/input/sjn2_11.45-12.00.mp4"
DESTINATION_VIDEO = "/Users/farhanfadillahr/Documents/data/Kerja/yolov8/assets/output/existing-5fps.avi"
CUSTOM_TRACK = "/Users/farhanfadillahr/Documents/data/Kerja/yolov8/assets/custom_track.yaml"


In [ ]:
import torch
import cv2
import numpy as np
import pathlib
import matplotlib.pyplot as plt
from ultralytics import YOLO
from collections import deque
from datetime import datetime

In [ ]:
# function for intersect
def intersect(A,B,C,D):
    return ccw(A,C,D) != ccw(B,C,D) and ccw(A,B,C) != ccw(A,B,D)

def ccw(A,B,C):
    return (C[1]-A[1]) * (B[0]-A[0]) > (B[1]-A[1]) * (C[0]-A[0])

def calculate_slope(x1, y1, x2, y2):
    # Hitung gradien dari garis yang didefinisikan oleh dua titik
    if x2 - x1 == 0:
        return None  # Gradien tak hingga
    return (y2 - y1) / (x2 - x1)

def calculate_intercept(x, y, slope):
    # Hitung perpotongan garis dengan sumbu y
    if slope is None:
        return None  # Garis vertikal
    return y - slope * x

def intersect(A, B, C, D):
    # Hitung gradien dan perpotongan garis dari line
    slope = calculate_slope(C[0], C[1], D[0], D[1])
    intercept = calculate_intercept(D[0], D[1] , slope)

    # Hitung nilai y yang diharapkan berdasarkan nilai x objek (x2 dan x3)
    y2_expected = slope * A[0] + intercept
    y3_expected = slope * B[0] + intercept

    # Periksa apakah objek melewati garis
    return (A[1] >= y2_expected and B[1] <= y3_expected) or (A[1] <= y2_expected and B[1] >= y3_expected)

def get_direction(point1, point2):
    direction_str = ""

    # calculate y axis direction
    if point1[1] > point2[1]:
        direction_str += "South"
    elif point1[1] < point2[1]:
        direction_str += "North"
    else:
        direction_str += ""

    return direction_str

In [ ]:
# line_warga = [(533, 607), (1012, 522)] # 1920 x 1080
line_warga = [(343, 387), (593, 317)] # 1280 x 720

obj_id = {}
ids = []

data_deque = {}
object_counter_s = {}
object_counter_n = {}
total_n = 0
total_s = 0

count = 0

In [ ]:
model = YOLO("assets/yolov8l.pt")
device = '0' if torch.cuda.is_available() else "cpu"
if device == '0':
  torch.cuda.set_device(0)

vs = cv2.VideoCapture(SOURCE_VIDEO)

size = (int(vs.get(3)), int(vs.get(4)))
fps = int(vs.get(5))
set_fps = 5
fps = fps//set_fps
# result = cv2.VideoWriter(DESTINATION_VIDEO, cv2.VideoWriter_fourcc(*'MJPG'), 1, (1280, 720))
# Video writer
video_writer = cv2.VideoWriter(DESTINATION_VIDEO,
                       cv2.VideoWriter_fourcc(*'mp4v'),
                       set_fps,
                       (1280, 720))

while True:
    before = datetime.now()

    ret, frame = vs.read()

    if not ret:
        break
    frame = cv2.resize(frame, (1280, 720)) # 1280 x 720 resolution

    classes = [2,3,5,7] # car, motorcycle, bus, truck
    results = model.track(frame, conf=0.1, iou=0.1, classes=classes, persist=True, tracker=CUSTOM_TRACK, verbose=False)
    # res_plotted = results[0].plot(line_width=2, labels=False)

    cv2.line(frame, line_warga[0], line_warga[1], (255, 84, 46), 3) # biru

    # Get the boxes and track IDs
    # boxes = results[0].boxes.xywh
    boxes = results[0].boxes.xyxy
    track_ids = results[0].boxes.id
    cls = results[0].boxes.cls
    confs = results[0].boxes.conf

    # remove tracked point from buffer if object is lost
    idx = []
    if track_ids != None:
        for i in track_ids:
            idx.append(int(i))
    for key in list(data_deque):
        if key not in idx:
            data_deque.pop(key)

    if track_ids != None:
        # looping get id and counting
        for box, id, cls, conf in zip(boxes, track_ids, cls, confs):
            # x, y, w, h = box
            # x = int(x)
            # y = int(y)
            # w = int(w)
            # h = int(h)
            # id = int(id)
            # conf = int(conf)

            x1, y1, x2, y2 = box
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)
            id = int(id)
            conf = int(conf)
            obj_name = model.names[int(cls)]
            cx, cy = int(x1+((x2-x1)/2)), int(y1+((y2-y1)/2))
            center = (cx, cy)

            if obj_name == 'car':
                cv2.rectangle(frame, (x1,y1), (x2,y2), (255,0,249), thickness=2) # magenta
            elif obj_name == 'motorcycle':
                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,255), thickness=2) # yellow
            elif obj_name == 'bus':
                cv2.rectangle(frame, (x1,y1), (x2,y2), (255,254,62), thickness=2) # cyan
            elif obj_name == 'truck':
                cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), thickness=2) # green

            if id not in data_deque:
                data_deque[id] = deque(maxlen= 64)
            data_deque[id].appendleft(center)

            if len(data_deque[id]) >= 2:
                direction = get_direction(data_deque[id][0], data_deque[id][1])
                if intersect(data_deque[id][0], data_deque[id][1], line_warga[0], line_warga[1]):
                    cv2.line(frame, line_warga[0], line_warga[1], (255, 255, 255), 3) # putih
                    if "South" in direction:
                        if obj_name not in object_counter_s:
                            object_counter_s[obj_name] = 1
                            total_n += 1
                        else:
                            object_counter_s[obj_name] += 1
                            total_n += 1

                    if "North" in direction:
                        if obj_name not in object_counter_n:
                            object_counter_n[obj_name] = 1
                            total_s += 1
                        else:
                            object_counter_n[obj_name] += 1
                            total_s += 1

    # resolusi : 1280 x 720
    cv2.line(frame, (535,35), (765,35), (85,45,255), 35)
    cv2.putText(frame, f'Kendaraan Keluar :', (525, 45), 0, 0.75, [255, 255, 255], thickness=2, lineType=cv2.LINE_AA)
    cv2.line(frame, (535, 70), (700, 70), [85, 45, 255], 35)
    cv2.putText(frame, f'Total : {total_n}', (525, 80), 0, 0.75, [255, 255, 255], thickness = 2, lineType = cv2.LINE_AA)

    cv2.line(frame, (895,35), (1125,35), (85,45,255), 35)
    cv2.putText(frame, f'Kendaraan Masuk :', (885, 45), 0, 0.75, [255, 255, 255], thickness=2, lineType=cv2.LINE_AA)
    cv2.line(frame, (895, 70), (1060, 70), [85,45,255], 35)
    cv2.putText(frame, f'Total : {total_s}', (885, 80), 0, 0.75, [255, 255, 255], thickness=2, lineType=cv2.LINE_AA)

    # cv2.imshow("test", frame)
    # cv2.waitKey(1)

    # count += 1
    count += fps
    vs.set(cv2.CAP_PROP_POS_FRAMES, count)
    print(f"resolusi : {frame.shape[1]} x {frame.shape[0]}")
    print("total frame : ", vs.get(7))
    after = datetime.now()
    print('waktu analytics 1 frame : ', (after-before))
    print("frame ke " + str(count))
    print('====================================================================================================')

    # result.write(frame)
    video_writer.write(frame)
    # cv2_imshow(frame)



    # key = cv2.waitKey(1)
    # if key == ord("q"):
    #   break

vs.release()
video_writer.release()
cv2.destroyAllWindows()